In [1]:
import sklearn
import pandas as pd 
import numpy as np
import gc
import time
from operator import itemgetter
from sklearn.model_selection import KFold
from collections import OrderedDict


path = "D:/kaggle/MovieLens/"
path2 = 'D:/kaggle/MovieLens/ml-latest-small/'
#data = pd.read_csv( path+"rating.csv", usecols=[0,1])
data = pd.read_csv( path2+"ratings.csv", usecols=[0,1])

In [2]:
print(data.shape)
data.head()

(100836, 2)


,userId,movieId
0,1,1
1,1,3
2,1,6
3,1,47
4,1,50


In [3]:
# 剔除稀疏和稠密的数据
temp = data.groupby(['movieId'],as_index=False)['userId'].count().rename(columns={'userId':"user_count"})
data = data.merge(temp, 'left', 'movieId')
data = data[(data['user_count'] > 3) & (data['user_count'] < 200)]
temp = data.groupby(['userId'],as_index=False)['movieId'].count().rename(columns={'movieId':"movie_count"})
data = data.merge(temp, 'left', 'userId')
data = data[ (data['movie_count'] > 3) & (data['movie_count'] < 800)]

In [4]:
data.nunique()

userId          594
movieId        4135
user_count      156
movie_count     237
dtype: int64

In [5]:
data.shape

(69637, 4)

In [6]:
kf = KFold(n_splits=8, shuffle=True, random_state=22)

for train_index, valid_index in kf.split(data):
    train = data.iloc[train_index]
    test = data.iloc[valid_index]
    user_items = {}
    for userid, group in train.groupby("userId"):
        user_items[userid] = set( group['movieId'])

    item_users = {}
    for movieid, group in train.groupby("movieId"):
        item_users[movieid] = set( group['userId'])
        
    test_user_items = {}
    for userid, group in test.groupby("userId"):
        test_user_items[userid] = set( group['movieId'])
    
    break

In [7]:
class randomWalk(object):
    def __init__(self, user_items, item_users, length=10, epoch=9, alpha=0.8):
        self.user_items = user_items
        self.item_users = item_users
        self.length = length
        self.epoch = epoch
        self.alpha =alpha
        self.pretrain = False
        
    def precommend(self):
        length = self.length
        self.user_recommend = {}
        for i, user in enumerate(self.user_items.keys()):
            if( i%50==0):
                print("%s users" % i)
            self.user_recommend[user] = self.PersonalRank( user, length)
        
        self.pretrain = True
        
    def recommend(self, user, length=10):
        if self.pretrain == False:
            return self.PersonalRank(user, length)
        else:
            return self.user_recommend[user]
            

    def PersonalRank(self, root, length):
        alpha = self.alpha
        is_user = {}
        is_user[root] = 1
        rank = OrderedDict()
        rank[root] = 1.0
        # 迭代10次基本上就收敛了
        for k in range(self.epoch):
            temp = OrderedDict()
            iter_set = set(is_user.keys())
            for i in iter_set:
                if is_user[i]:
                    neighbors = user_items[i]
                else:
                    neighbors = self.item_users[i]
                temp[root] = temp.get(root, 0) + (1 - alpha) * rank.get(i,0)
                for j in neighbors:
                    if not j in is_user:
                        if is_user[i]:
                            is_user[j] = 0
                        else:
                            is_user[j] = 1
                    temp[j] = temp.get(j, 0) + alpha * rank.get(i,0) /  len(neighbors)

            #print(np.sum([i for i in temp.values()]))
            rank = temp

        ret = {}
        for i, key in enumerate(rank):
            if i>1000:
                break
            else:
                if is_user[key] == 0 and not key in user_items[root]:
                    ret[key] = rank[key]
        return sorted(ret.items(), key=lambda d:d[1], reverse=True)[0:length]

In [8]:
model = randomWalk( user_items, item_users)
model.precommend()

0 users
50 users
100 users
150 users
200 users
250 users
300 users
350 users
400 users
450 users
500 users
550 users


In [9]:
def Recall_Precision(obj, test, topN):
    hit = 0
    n_recall = 0
    n_precision = 0
    for user in test.keys():
        true = test[user]
        rank = obj.recommend(user, length=topN)
        rank = {it[0] for it in rank}
        hit += len(true & rank)
        n_recall += len(true)
        n_precision += topN
    return hit / n_recall, hit / n_precision

In [11]:
Recall_Precision(model,test_user_items, 10)

(0.062377943710511204, 0.09250425894378195)